In [ ]:
!git clone https://github.com/minjae0501/LSTM_Lecture.git

Cloning into 'LSTM_Lecture'...
remote: Enumerating objects: 546, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 546 (delta 2), reused 6 (delta 0), pack-reused 536
Receiving objects: 100% (546/546), 270.11 MiB | 15.01 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
!nvidia-smi

Sat Feb  3 08:12:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              44W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# YOLO_Train

In [ ]:
data_dir = '/content/LSTM_Lecture/data/YOLO_data'
model = 'yolov8n.pt'  # pretrained yolov8n

classes = ['block', 'hand']  # 라벨명
nc = len(classes)  # number of classes
yaml_file = 'data.yaml'  # yaml 파일명



### ULTRALYTICS SETUP

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.9 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 26.9/166.8 GB disk)


In [ ]:
from ultralytics import YOLO
import os
import yaml

In [ ]:
# python: dictionary <----------> yaml
data = {
    'train': data_dir + '/train/',
    'val': data_dir + '/vali/',
    'test': data_dir + '/test/',
    'nc': nc,
    'names': classes
}

os.makedirs(data_dir, exist_ok = True) # directory to save

with open(data_dir + '/'+ yaml_file, 'wt') as fw:
    yaml.dump(data, fw)

In [ ]:
with open(data_dir + '/' + yaml_file, 'rt') as fr:
    d = yaml.safe_load(fr)
    print(type(d))
    print(d)

<class 'dict'>
{'names': ['block', 'hand'], 'nc': 2, 'test': '/content/LSTM_Lecture/data/YOLO_data/test/', 'train': '/content/LSTM_Lecture/data/YOLO_data/train/', 'val': '/content/LSTM_Lecture/data/YOLO_data/vali/'}


### Model Training

In [ ]:
import torch

if torch.cuda.is_available() == True:
    device = 'cuda:0'
else:
    device = 'cpu'

print(device)

cuda:0


In [ ]:
# BASIC CONFIG (필요한 파라미터 값 여기서 설정)

training_epoch = 50
patience = 30  # 지정한 epoch 만큼 성능이 개선이 안되면 조기종료.
batch = 32  # batch size (dafault)
imgsz = 640  # input batch size (default)
optimizer = 'Adam'

In [ ]:
model = YOLO(model).to(device)  # pretrained yolov8n

model.train(data = data_dir + '/' +yaml_file,
            epochs = training_epoch,
            patience = patience, # 지정한 epoch 만큼 성능이 개선이 안되면 조기종료.
            batch = batch, # batch size (dafault)
            imgsz = imgsz, # input batch size (default)
            optimizer = optimizer)

100%|██████████| 6.23M/6.23M [00:00<00:00, 362MB/s]


engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/LSTM_Lecture/data/YOLO_data/data.yaml, epochs=50, time=None, patience=30, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, opti

100%|██████████| 755k/755k [00:00<00:00, 130MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/LSTM_Lecture/data/YOLO_data/train/labels... 239 images, 0 backgrounds, 0 corrupt: 100%|██████████| 239/239 [00:00<00:00, 1273.44it/s]

...

                   all         20         40      0.996          1      0.995      0.711



50 epochs completed in 0.023 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.9 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 10.34it/s]


                   all         20         40      0.996          1      0.995      0.712
                 block         20         20      0.995          1      0.995      0.755
                  hand         20         20      0.997          1      0.995       0.67
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f2df99fc370>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

### Model Evaluation

In [ ]:
YOLO_best_model_path = '/content/runs/detect/train/weights/best.pt'

YOLO_best_model = YOLO(YOLO_best_model_path).to(device)
metrics = YOLO_best_model.val()

for label, ap in zip(classes, metrics.box.maps):
    print(label, ':', ap)

Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/LSTM_Lecture/data/YOLO_data/vali/labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.38it/s]


                   all         20         40      0.996          1      0.995      0.707
                 block         20         20      0.995          1      0.995      0.745
                  hand         20         20      0.997          1      0.995       0.67
Speed: 0.2ms preprocess, 24.9ms inference, 0.0ms loss, 6.6ms postprocess per image
Results saved to runs/detect/val
block : 0.7449274010346378
hand : 0.6696009407259408
